In [1]:
#import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

import pickle 
#import mglearn
import time

from nltk.tokenize import TweetTokenizer # doesn't split at apostrophes
import nltk
from nltk import Text
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize  
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

In [2]:
#importing the dataset
df = pd.read_csv('NLP - rusentiment_preselected_posts.csv')

In [3]:
df.head(3)

,label,text
0,neutral,Прорвём информационную блокаду изнутри.
1,negative,"Никогда у меня не будет ""одного приложения для..."
2,skip,"Кури-и тебя не укусит злая собака, потому что ..."


In [4]:
df['label'].unique()

array(['neutral', 'negative', 'skip', 'positive', 'speech'], dtype=object)

In [5]:
codes = {'negative':-1, 'positive':1, 'neutral':0}

In [6]:
#Indicating labels as numbers for ML model
df['label'] = df['label'].map(codes)

In [7]:
df.head(3)

,label,text
0,0.0,Прорвём информационную блокаду изнутри.
1,-1.0,"Никогда у меня не будет ""одного приложения для..."
2,NaN,"Кури-и тебя не укусит злая собака, потому что ..."


In [8]:
from nltk.corpus import stopwords
stop = stopwords.words("Russian")
df['text'] = df['text'].apply(lambda x: ','.join([item for item in x.split('\t') if item not in stop]))

In [9]:
#Removing null values
df.isnull().sum()
df.dropna(inplace=True)
df.head()

,label,text
0,0.0,Прорвём информационную блокаду изнутри.
1,-1.0,"Никогда у меня не будет ""одного приложения для..."
3,0.0,"Есть 3 типа людей:\nУмные, которые делают все ..."
4,0.0,мегафон чет накрыло
7,0.0,Шах и мат.


In [15]:
# Initialize a CountVectorizer object: count_vectorizer
count_vec = CountVectorizer(stop_words=stopwords.words('russian'), analyzer='word', encoding='KOI8-R',
                            ngram_range=(1, 1), max_df=0.5, min_df=1, max_features=None)

In [16]:
count_train = count_vec.fit(df.text)
bag_of_words = count_vec.transform(df.text)
print(bag_of_words[:1])

  (0, 1552)	1
  (0, 6651)	1
  (0, 6793)	1
  (0, 15204)	1


In [17]:
print("text size:\n {}".format(len(count_vec.get_feature_names())))
print("text content:\n {}".format(count_train.vocabulary_))

text size:
 21901
text content:
 {'прорвём': 15204, 'информационную': 6793, 'блокаду': 1552, 'изнутри': 6651, 'одного': 11361, 'приложения': 14778, 'топку': 19275, 'ущербные': 20386, 'универсальности': 20095, 'типа': 19192, 'людей': 8674, 'умные': 20062, 'которые': 7820, 'делают': 4323, 'сразу': 18217, 'откладывают': 11891, 'доделывают': 4674, 'сначала': 17679, 'впадлу': 2874, 'поздно': 13435, 'мегафон': 8969, 'чет': 21244, 'накрыло': 9892, 'шах': 21443, 'мат': 8897, 'просто': 15265, 'дрожь': 4940, 'бросает': 1819, 'такими': 18914, 'тупыми': 19533, 'хохлянских': 20889, 'фразочек': 20617, 'офигеваю': 12146, 'школярского': 21532, 'голоса': 3714, 'этих': 21754, 'кортавых': 7762, 'фраз': 20613, 'вообще': 2768, 'херею': 20765, 'косяк': 7792, 'соскучилась': 17988, 'тебе': 19037, 'плсторел': 12933, 'бляять': 1570, 'забей': 5563, 'ужеееееее': 19878, 'нам': 9904, 'мало': 8775, 'лет': 8325, 'горло': 3787, 'сыты': 18859, 'этим': 21752, 'миром': 9234, 'хочу': 20896, 'выходить': 3378, 'улицу': 1998

In [18]:
# Initialize a TfidfVectorizer
tf = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word',
                     stop_words=stopwords.words('russian'), encoding='KOI8-R', max_features=None)
txt_fitted = tf.fit(df.text)
txt_transformed = txt_fitted.transform(df.text)
print ("The text: ", df.text[:5])
print ("The txt_transformed: ", txt_transformed[:5])

The text:  0             Прорвём информационную блокаду изнутри. 
1    Никогда у меня не будет "одного приложения для...
3    Есть 3 типа людей:\nУмные, которые делают все ...
4                                  мегафон чет накрыло
7                                           Шах и мат.
Name: text, dtype: object
The txt_transformed:    (0, 15204)	9.667680024690174
  (0, 6793)	9.667680024690174
  (0, 6651)	8.281385663570283
  (0, 1552)	8.974532844130227
  (1, 20386)	9.667680024690174
  (1, 20095)	9.667680024690174
  (1, 19275)	9.667680024690174
  (1, 14778)	8.569067736022063
  (1, 11361)	6.532185808761024
  (2, 20062)	17.138135472044127
  (2, 19192)	7.028622695074915
  (2, 18217)	6.266482643028018
  (2, 17679)	6.671947751136183
  (2, 13435)	6.723241045523733
  (2, 11891)	9.667680024690174
  (2, 8674)	5.363614931486004
  (2, 7820)	10.83836956528163
  (2, 4674)	9.667680024690174
  (2, 4323)	7.365094931696128
  (2, 2874)	8.569067736022063
  (3, 21244)	8.058242112256073
  (3, 9892)	9.66768002

In [19]:
X = txt_transformed.toarray()

In [20]:
X.shape

(5812, 21901)

In [22]:
y = df.label.values
y.shape

(5812,)

### Machine Learning models

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [24]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None)

In [25]:
classifier.predict(X_test)
classifier.score(X_test, y_test)

0.4032674118658641

In [28]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='lbfgs', alpha=0.1,
                    hidden_layer_sizes=(100, 100), random_state=1, learning_rate_init=1, activation='relu')

In [29]:
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
mlp.score(X_test, y_test)

0.5227858985382631

In [37]:
from sklearn.linear_model import SGDClassifier
sgdc = SGDClassifier()
sgdc.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [38]:
sgdc.predict(X_test)
sgdc.score(X_test, y_test)

0.5167669819432502

In [30]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
ONA = OneVsRestClassifier(LinearSVC(random_state=0))

In [31]:
ONA.fit(X_train, y_train)
ONA.predict(X_test)
ONA.score(X_test, y_test)

0.5374032674118658

In [39]:
from sklearn.ensemble import RandomForestClassifier
ranfor = RandomForestClassifier()
ranfor.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [46]:
y_pred = ranfor.predict(X_test)
ranfor.score(X_test, y_test)

0.5580395528804815

In [47]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 51, 207,  19],
       [ 59, 505,  43],
       [ 18, 168,  93]], dtype=int64)